# Zeitentwicklung eines freien Teilchens

## Vorbereitung

Führen sie zuerst die folgende Zelle aus, um die Simulation vorzubereiten. Sie können diese danach mit dem Pfeil links einklappen. 

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class Schroedinger_Equation:
    def __init__(self, hbar=1, m=1):
        self.hbar = hbar
        self.m = m
        
        
    def initialize_grid(self, potential_type, tmin, tmax, Nt, xmin, xmax, Nx, k0=1, d=1, p=0.5):
        self.t_grid = np.linspace(tmin, tmax, Nt)
        self.dt = self.t_grid[1] - self.t_grid[0]
        
        self.dx=(xmax-xmin)/(Nx)
        self.x_grid = np.arange(xmin,xmax,self.dx)
        
        self.dk = 2*np.pi/(xmax-xmin)
        self.k_grid = np.arange(-self.dk*Nx/2,self.dk*Nx/2,self.dk)
        
        self.Nt = Nt # Anzahl der Zeitschritte
        self.Nx = Nx
        self.Nk = Nx
        self.xmin = xmin
        self.xmax = xmax
        self.kmin = self.k_grid[0]
        self.kmax = self.k_grid[-1]
        
        self.fourier_setup()
        
        self.psi_x = np.zeros((self.Nt,self.Nx),dtype=complex)
        self.psi_k = np.zeros((self.Nt,self.Nx),dtype=complex)
        
        
        if potential_type == 'Glauber_Zustand':
            q=0   
            self.psi_x[0] = 0.5 * np.exp(1j*self.x_grid*p/self.hbar) * np.exp(-0.5*(self.x_grid-q)**2)  # No Pi factor?
            self.psi_k[0] = self.discrete_fourier_trafo(self.psi_x[0])
        else:
            self.psi_k[0] = np.exp( -(self.k_grid-k0)**2 * d**2 ) # Vary d !!!
            self.psi_x[0] = self.inv_discrete_fourier_trafo(self.psi_k[0])
        
        
    def fourier_setup(self):
        self.discrete_fourier_trafo_setup = np.zeros((self.Nk,self.Nx),dtype=complex)
        for i in range(0,self.Nk):
            for j in range(0,self.Nx):
                self.discrete_fourier_trafo_setup[i,j] = self.dx / np.sqrt(2*np.pi) * np.exp(-1j*self.k_grid[i]*self.x_grid[j])
        
        self.inv_discrete_fourier_trafo_setup = np.zeros((self.Nx,self.Nk),dtype=complex)
        for j in range(0,self.Nx):
            for i in range(0,self.Nk):
                self.inv_discrete_fourier_trafo_setup[j,i] = self.dk / np.sqrt(2*np.pi) * np.exp(1j*self.k_grid[i]*self.x_grid[j])
        
        
    def discrete_fourier_trafo(self,psi_x):
        psi_k = self.discrete_fourier_trafo_setup @ psi_x
        return psi_k
    
    def inv_discrete_fourier_trafo(self,psi_k):
        psi_x = self.inv_discrete_fourier_trafo_setup @ psi_k
        return psi_x
    
    def calculate_rho(self,psi):
        rho = np.abs(psi)**2
        return rho
    
    def split_operator_method(self, potential_type = 'Freies_Teilchen', v=0):
        k_propagate_half = np.exp( -1j * (self.hbar/(2.*self.m)) * self.k_grid**2 * self.dt / 2.)
        k_propagate = k_propagate_half * k_propagate_half #freies Teilchen
        
        if potential_type == 'Freies_Teilchen':
            for t in range(1, self.Nt):
                self.psi_k[t] = self.psi_k[t-1] * k_propagate
                self.psi_x[t] = self.inv_discrete_fourier_trafo( self.psi_k[t] )
                
        elif potential_type == 'Barriere' or potential_type=='Glauber_Zustand':
            self.init_v(potential_type, v)
            x_propagate = np.exp(-1j*self.v_vector * self.dt / self.hbar)
            for t in range(1, self.Nt): #tqdm( range(1, self.Nt) ):
                psi_k_temporary = self.psi_k[t-1] * k_propagate_half
                psi_x_temporary = self.inv_discrete_fourier_trafo(psi_k_temporary) * x_propagate
                self.psi_k[t] = self.discrete_fourier_trafo(psi_x_temporary) * k_propagate_half
                self.psi_x[t] = self.inv_discrete_fourier_trafo(self.psi_k[t])
                
                
    def init_v(self, potential_type, v0, x0=40, b=2): #Gauss Potential
        if potential_type == 'Barriere':
            self.v_vector = v0*np.exp(-0.5*((self.x_grid-x0)/b)**2)
        elif potential_type == 'Glauber_Zustand':
            self.v_vector = 0.5 * self.x_grid**2
            
    
    def plot_psi(self, t, potential_type):
        rho_x = self.calculate_rho(self.psi_x)
        
        fig = make_subplots(rows=3, cols=1,
                            subplot_titles=['Wellenfunktion im Ortsraum', 'Wellenfunktion im Impulsraum',
                                            'Aufenthaltswahrscheinlichkeit im Ortsraum'],
                            )
        
        for time in range(self.Nt):
            fig.add_trace(go.Scatter(x=self.x_grid, y=np.real(self.psi_x[time]), visible=False, 
                                     name=r'$\text{Re}\left(\psi\left(x,t\right)\right)$'), 1, 1)
            fig.add_trace(go.Scatter(x=self.x_grid, y=np.imag(self.psi_x[time]), visible=False, 
                                     name=r'$\text{Im}\left(\psi\left(x,t\right)\right)$'), 1, 1)
    
            fig.add_trace(go.Scatter(x=self.k_grid, y=np.real(self.psi_k[time]), visible=False, 
                                     name=r'$\text{Re}\left(\tilde{\psi}\left(k,t\right)\right)$'), 2, 1)
            fig.add_trace(go.Scatter(x=self.k_grid, y=np.imag(self.psi_k[time]), visible=False, 
                                     name=r'$\text{Im}\left(\tilde{\psi}\left(k,t\right)\right)$'), 2, 1)
            
            
            fig.add_trace(go.Scatter(x=self.x_grid, y=rho_x[time], visible=False, 
                                     name=r'$|\psi(x,t)|^2$'), 3, 1)
    
        fig.data[0].visible = True
        fig.data[1].visible = True
        fig.data[2].visible = True
        fig.data[3].visible = True
        fig.data[4].visible = True

        if potential_type=='Barriere' or potential_type=='Glauber_Zustand': #Plot potential
            fig.add_trace(go.Scatter(x=self.x_grid, y=self.v_vector, visible=True, 
                                     name=r'$V(x)$'), 3, 1)      
        
        steps = []
        for i in range(0, 5*self.Nt, 5):
            if potential_type=='Barriere' or potential_type=='Glauber_Zustand': # Potential always visible
                step = dict(method="update", args=[{"visible": [False] * (5*self.Nt+1)}], label=str(i//5))
                step["args"][0]["visible"][i:i+5] = [True,True,True,True,True]
                step["args"][0]["visible"][-1] = True 
            else:
                step = dict(method="update", args=[{"visible": [False] * 5*self.Nt}], label=str(i//5))
                step["args"][0]["visible"][i:i+5] = [True,True,True,True,True]
            steps.append(step)
            
        slider = [dict(active=0, steps=steps, currentvalue={"prefix": "t="})]

        fig.update_layout(sliders=slider, width=1000, height=800)
        
        if potential_type=='Freies_Teilchen':
            fig.update_xaxes(range=[-5, 15])
            #fig.update_yaxes(range=[-1, 1])
            
        elif potential_type=='Barriere':
            fig.update_xaxes(range=[-50, 90], row=1, col=1)
            fig.update_xaxes(range=[-10, 10], row=2, col=1)
            fig.update_xaxes(range=[-50, 90], row=3, col=1)
            #fig.update_yaxes(range=[-1, 1])
            
        elif potential_type=='Glauber_Zustand':
            fig.update_xaxes(range=[-5, 5])
            fig.update_xaxes(range=[-10, 10], row=2, col=1)
            #fig.update_yaxes(range=[-1, 1])
        
        psi_x_min = min([np.real(self.psi_x).min(),np.imag(self.psi_x).min()])
        psi_x_max = max([np.real(self.psi_x).max(),np.imag(self.psi_x).max()])
        fig.update_yaxes(range=[psi_x_min-0.02, psi_x_max+0.02], row=1, col=1)
        
        psi_k_min = min([np.real(self.psi_k).min(),np.imag(self.psi_k).min()])
        psi_k_max = max([np.real(self.psi_k).max(),np.imag(self.psi_k).max()])
        fig.update_yaxes(range=[psi_k_min-0.02, psi_k_max+0.02], row=2, col=1)
        
        fig.update_yaxes(range=[rho_x.min()-0.02, rho_x.max()+0.02], row=3, col=1)
        
        fig.show()
  
        
    def solve(self, potential_type='Freies_Teilchen' ,v0=0, tmin=None, tmax=None, Nt=None, xmin=None, xmax=None, Nx=None, k0=None, d=None, p=0.5):
        # Check potential_type Input!!!
        
        if tmin==None or tmax==None or Nt==None:
            if potential_type=='Freies_Teilchen':
                tmin = 0
                tmax = 5
                Nt = 50
            elif potential_type=='Barriere':
                tmin = 0
                tmax = 100
                Nt = 100
            elif potential_type=='Glauber_Zustand':
                tmin = 0
                tmax = 20
                Nt = 100
        else:
            assert tmin < tmax, "Min<Max required"
            assert Nt>0, 'Nt>0 required'

        if xmin==None or xmax==None or Nx==None:
            if potential_type=='Freies_Teilchen':
                xmin = -30
                xmax = 30
                Nx = 2**10
            elif potential_type=='Barriere':
                xmin = -100
                xmax = 100
                Nx = 2**10
            elif potential_type=='Glauber_Zustand':
                xmin = -6
                xmax = 6
                Nx = 2**10
        else:
            assert xmin < xmax, "Min<Max required"
            assert Nx>0, 'Nt>0 required'
            
        if k0==None:
            if potential_type=='Freies_Teilchen':
                k0 = 1
            elif potential_type=='Barriere':
                k0 = 2
        if d==None:
            if potential_type=='Freies_Teilchen':
                d = 1
            elif potential_type=='Barriere':
                d = 2
        
        
        self.initialize_grid(potential_type, tmin, tmax, Nt, xmin, xmax, Nx, k0, d, p)
        
        self.split_operator_method(potential_type, v0)
        
        self.plot_psi(0, potential_type)

## Simulation

Durch das ausführen der nächsten Zeile starten sie die Simulation. Um den Parameter $d$ zu variieren, den gewünschten Wert in der nächsten Zelle eingeben und diese erneut ausführen. 

In [ ]:
schroedinger_class = Schroedinger_Equation()
schroedinger_class.solve('Freies_Teilchen', d = 1)

## Schritt für Schritt Berechnung der Animation

In [ ]:
hbar = 1
m = 1

### Initialisierung der diskreten 1D Koordinatensysteme

In [ ]:
t_min = 0
t_max = 4
Nt = 40
dt = (t_max-t_min)/(Nt)
t_grid = np.arange(t_min,t_max,dt)

In [ ]:
x_min = -30
x_max = 30
Nx = 2**10
dx = (x_max-x_min)/(Nx)
x_grid = np.arange(x_min,x_max,dx)

Erklärung für Umwandlung in Impulsraum

In [ ]:
Nk = Nx
dk = dx/(2*np.pi)#2*np.pi/(x_max-x_min) !!! Klären!!!
k_grid = np.arange(-dk*Nk/2,dk*Nk/2,dk)

### Definition der diskreten Fourier-Transformation

Die diskrete Fourier-Transformation
$\tilde{\Psi}(k_{j}) = \frac{dx}{\sqrt{2\pi}}\sum_{i}e^{-ik_jx_i} \Psi(x_{i}) $ 
und ihre Inverse

wird mithilfe einer Matrix-Vektor Multiplikation der Form

$
\begin{pmatrix}\tilde{\Psi}(k_1) \\ : \\ \tilde{\Psi}(k_N) \end{pmatrix} = 
\begin{pmatrix}\frac{dx}{\sqrt{2\pi}}e^{-ik_1x_1} & .. & \frac{dx}{\sqrt{2\pi}}e^{-ik_1x_N} \\ 
: & & : \\
\frac{dx}{\sqrt{2\pi}}e^{-ik_Nx_1} & .. & \frac{dx}{\sqrt{2\pi}}e^{-ik_Nx_N} \end{pmatrix}
\begin{pmatrix}\Psi(x_1) \\ : \\ \Psi(x_N) \end{pmatrix}
$

berechnet. Analog wird die inverse diskrete Fourier-Transformation 
$\Psi(x_{j}) = \frac{dx}{\sqrt{2\pi}}\sum_{i}e^{ik_ix_j} \tilde{\Psi}(k_{i}) $ 
durchgeführt.

In [ ]:
discrete_fourier_trafo_setup = np.zeros((Nk,Nx),dtype=complex)
for i in range(0,Nk):
    for j in range(0,Nx):
        discrete_fourier_trafo_setup[i,j] = dx / np.sqrt(2*np.pi) * np.exp(-1j*k_grid[i]*x_grid[j]) 
        # Die imaginare Einheit i wird in Python mit 1j bezeichnet

In [ ]:
def discrete_fourier_trafo(psi_x):
    psi_k = discrete_fourier_trafo_setup @ psi_x # Matrix-Vektor Multiplikation
    return psi_k

In [ ]:
inv_discrete_fourier_trafo_setup = np.zeros((Nx,Nk),dtype=complex)
for j in range(0,Nx):
    for i in range(0,Nk):
        inv_discrete_fourier_trafo_setup[j,i] = dk / np.sqrt(2*np.pi) * np.exp(1j*k_grid[i]*x_grid[j])

In [ ]:
def inv_discrete_fourier_trafo(psi_k):
    psi_x = inv_discrete_fourier_trafo_setup @ psi_k # Matrix-Vektor Multiplikation
    return psi_x

### Wellenfunktion initialisieren

Die Wellenfunktion im Impulsraum zum Zeitpunkt $t=0$ wird als das Gaußsche Wellenpaket

$\tilde{\Psi}(k)= e^{-(k-k_0)^2/d^2}$

definiert, und die Wellenfunktion im Ortsraum zum Zeitpunkt $t=0$ wird mithilfe der inversen diskreten Fourier-Transformation berechnet.
Hierbei gibt $k_0$ den Mittelwert und $d$ die Breite des Wellenpaketes an.

In [ ]:
k0 = 1
d = 1
psi_k = np.zeros((Nt,Nx),dtype=complex)
psi_x = np.zeros((Nt,Nx),dtype=complex)

In [ ]:
psi_k[0] = np.exp( -(k_grid-k0)**2 / d**2 )
psi_x[0] = inv_discrete_fourier_trafo(psi_k[0])

### Zeitentwicklung

Für jeden Zeitschritt wird $\tilde{\Psi}(k,t)$ mithilfe des Zeitentwicklungsoperators im Impulsraum berechnet:

$\tilde{\Psi}(k,t) = e^{-\frac{i}{\hbar}\frac{\hbar^2 k^2}{2 m}\Delta t} \: \tilde{\Psi}(k,t-1)$

$\Psi(x,t)$ wird danach wiederum mithilfe der inversen diskreten Fourier-Transformation berechnet.

In [ ]:
k_propagate = np.exp( -1j * (hbar/(2.*m)) * k_grid**2 * dt)

In [ ]:
for t in range(1, Nt):
    psi_k[t] = psi_k[t-1] * k_propagate
    psi_x[t] = inv_discrete_fourier_trafo( psi_k[t] )

### Wahrscheinlichkeitsdichte

Berechnung der Wahrscheinlichkeitsdichte im Ortsraum:

$\rho(x,t) = |\Psi(x,t)|^2$ 

In [ ]:
rho_x = np.abs(psi_x)**2

### Plotten

In [ ]:
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=['Wellenfunktion im Ortsraum', 'Wellenfunktion im Impulsraum', 'Aufenthaltswahrscheinlichkeit im Ortsraum'],
                    )

for time in range(Nt):
    fig.add_trace(go.Scatter(x=x_grid, y=np.real(psi_x[time]), visible=False, 
                             name=r'$\text{Re}\left(\psi\left(x,t\right)\right)$'), 1, 1)
    fig.add_trace(go.Scatter(x=x_grid, y=np.imag(psi_x[time]), visible=False, 
                             name=r'$\text{Im}\left(\psi\left(x,t\right)\right)$'), 1, 1)
    
    fig.add_trace(go.Scatter(x=k_grid, y=np.real(psi_k[time]), visible=False, 
                             name=r'$\text{Re}\left(\tilde{\psi}\left(k,t\right)\right)$'), 2, 1)
    fig.add_trace(go.Scatter(x=k_grid, y=np.imag(psi_k[time]), visible=False, 
                             name=r'$\text{Im}\left(\tilde{\psi}\left(k,t\right)\right)$'), 2, 1)
    
    fig.add_trace(go.Scatter(x=x_grid, y=rho_x[time], visible=False, 
                             name=r'$|\psi(x,t)|^2$'), 3, 1)

fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.data[4].visible = True

steps = []
for i in range(0, 5*Nt, 5):
    step = dict(method="update", args=[{"visible": [False] * 5*Nt}], label=str(i//2))
    step["args"][0]["visible"][i:i+5] = [True,True,True,True,True]
    steps.append(step)

slider = [dict(active=0, steps=steps, currentvalue={"prefix": "t="})]
fig.update_layout(sliders=slider, width=1000, height=800)

fig.update_xaxes(range=[-5, 15])
fig.update_xaxes(range=[-dk*Nk/2, dk*Nk/2], row=2, col=1)

psi_x_min = min([np.real(psi_x).min(),np.imag(psi_x).min()])
psi_x_max = max([np.real(psi_x).max(),np.imag(psi_x).max()])
fig.update_yaxes(range=[psi_x_min-0.02, psi_x_max+0.02], row=1, col=1)

psi_k_min = min([np.real(psi_k).min(),np.imag(psi_k).min()])
psi_k_max = max([np.real(psi_k).max(),np.imag(psi_k).max()])
fig.update_yaxes(range=[psi_k_min-0.02, psi_k_max+0.02], row=2, col=1)

fig.update_yaxes(range=[rho_x.min()-0.02, rho_x.max()+0.02], row=3, col=1)


fig.show()